In [1]:
#x01_populationStatistics

In [1]:
from chewc.callback import *
from chewc.policy import *
from chewc.sim import *
from chewc.lab import *
from chewc.config import *

In [2]:
import matplotlib.pyplot as plt
import uuid
import pdb
import torch
from matplotlib.animation import FuncAnimation
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import gymnasium as gym
import numpy as np
device='cpu'
from torch.utils.tensorboard import SummaryWriter



import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import CallbackList


In [3]:
#| export

import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import numpy as np
from chewc.sim import *
import torch

class SelectionIntensityEnvironment(gym.Env):
    def __init__(self, SP, config):
        super(SelectionIntensityEnvironment, self).__init__()
        self.SP = SP
        self.config = config  # Store the config
        self.current_generation = 0
        self.max_generations = SP.max_generations
        # Get action space bounds from config, with defaults if not provided
        
        self.action_low = config.get('action_low', 0.05)
        self.action_high = config.get('action_high', 0.95)
        
        self.action_space = gym.spaces.Box(
            low=np.array([self.action_low]), 
            high=np.array([self.action_high]), 
            dtype=np.float32
        )
        
        # Update observation space based on config
#         obs_config = config['observation_config']['remaining_proportion']
        self.observation_space = gym.spaces.Box(
            low=np.array([0, 0]),  # Lower bounds for remaining proportion and genetic variance
            high=np.array([1, np.inf]),  # Upper bounds (genetic variance can be any positive number)
            shape=(2,),
            dtype=np.float32
        )
        # logging
        self.action_values = []
        self.genetic_variance = []
        self.max_breeding_values = []
        self.final_generations = []
        self.episode_count = 0
        self.rewards = []
        self.episode_reward = 0
        
        #config
        self.config =config
        
    def _get_obs(self):
        remaining_proportion = 1 - (self.current_generation / self.max_generations)
        genetic_variance = self.population.breeding_values.var().cpu().item()
        return np.array([remaining_proportion, genetic_variance], dtype=np.float32)


    def _get_info(self):
        return {
            "max_phenotype": self.population.breeding_values.max().cpu().item(),
            "genetic_variance": self.population.breeding_values.var().cpu().item(),
            "current_generation": self.current_generation
        }

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.population = self.SP.founder_pop
        self.phenotype = phenotype(self.population, self.SP.T, self.SP.h2)
        self.current_generation = 0
        self.episode_reward = 0
        observation = self._get_obs()
        info = self._get_info()
        
        return observation, info

    def step(self, action):
        
        # Map the action from [-1, 1] to [action_low, action_high]
        action = scale_values(action, to_range=(self.action_low, self.action_high))

        total_selected = max((2,int(action * self.population.size)))
        selected = torch.topk(self.population.phenotypes, total_selected).indices
        self.population = create_pop(self.SP.G, random_crosses(self.population.haplotypes[selected], self.SP.pop_size))
        self.phenotype = phenotype(self.population, self.SP.T, self.SP.h2)
        self.current_generation += 1

        observation = self._get_obs()
        info = self._get_info()
        info['normalized_action'] = action
        
        terminated = self.current_generation > self.SP.max_generations
        #REWARD
        if self.config.get('sparse_reward', False):  # Use .get() with a default value
            reward = 0 if not terminated else float(self.population.breeding_values.max())
        else:
            reward = float(self.population.breeding_values.max())
        self.episode_reward += reward

        if terminated:
            info['final_generation'] = {
            "max_phenotype": self.population.breeding_values.max().cpu().item(),
            "genetic_variance": self.population.breeding_values.var().cpu().item(),
            "current_generation": self.current_generation
            }

        return observation, reward, bool(terminated), False, info

import torch as th
import torch.nn as nn
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.policies import ActorCriticPolicy

class CustomFeatureExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.spaces.Box, features_dim: int = 64):
        super(CustomFeatureExtractor, self).__init__(observation_space, features_dim)
        
        # For now, a simple linear layer for the scalar input
        self.linear = nn.Sequential(
            nn.Linear(observation_space.shape[0], features_dim),
            nn.ReLU()
        )

    def forward(self, observations: th.Tensor) -> th.Tensor:
        return self.linear(observations)

class CustomPolicy(ActorCriticPolicy):
    def __init__(self, *args, **kwargs):
        super(CustomPolicy, self).__init__(
            *args,
            **kwargs,
            features_extractor_class=CustomFeatureExtractor,
            features_extractor_kwargs=dict(features_dim=64)
        )

In [7]:
FE = CustomFeatureExtractor(env.observation_space)
mock_observation = th.tensor([[0.5]], dtype=th.float32)  # Single scalar input
FE(mock_observation)

tensor([[0.2324, 0.2336, 0.0000, 1.2855, 0.4397, 0.2425, 0.0000, 0.7930, 0.0579,
         0.0000, 0.7679, 0.0000, 0.7645, 0.0000, 0.8097, 0.3772, 0.4757, 0.0000,
         0.0000, 0.0550, 0.7841, 0.0000, 0.0000, 0.0000, 0.7531, 0.2036, 0.1383,
         0.0000, 0.3655, 0.2128, 0.0000, 0.0000, 0.2283, 0.0000, 0.0197, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.1436, 0.0000, 0.0000, 0.0000, 0.0000,
         0.2302, 0.0000, 0.0000, 0.0000, 0.0000, 0.4654, 0.9486, 0.0000, 0.6903,
         1.1337, 0.1533, 0.5235, 0.0000, 0.5588, 0.8903, 0.0000, 0.0000, 0.0000,
         0.0000]], grad_fn=<ReluBackward0>)

In [8]:
env = create_simulation()
env.reset()

(array([1.], dtype=float32),
 {'max_phenotype': 5.6667561531066895,
  'genetic_variance': 1.000000238418579,
  'current_generation': 0})

In [9]:
model = PPO(CustomPolicy, env, verbose=1)

model.learn(total_timesteps=100)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 11       |
|    ep_rew_mean     | 106      |
| time/              |          |
|    fps             | 746      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------


In [ ]:

def create_simulation(config=None):
    if config is None:
        config = get_default_config()
    
    seed = config['seed']
    set_seed(seed)
    
    G = Genome(config['n_chr'], config['n_loci'], seed=seed)
    
    founder_haplotypes = np.load('../nbs/data/g2f_ch10.npy')
    random_parent_indices = np.random.choice(founder_haplotypes.shape[0], config['n_parents'], replace=False)
    random_loci_indices = np.random.choice(founder_haplotypes.shape[2], config['n_loci'], replace=False)
    
    founder_haplotypes = founder_haplotypes[random_parent_indices,:,:]
    founder_haplotypes = founder_haplotypes[:,:,random_loci_indices]
    founder_haplotypes = torch.tensor(founder_haplotypes).unsqueeze(2)
    
    inbred_founders = create_pop(G, founder_haplotypes)
    f1 = create_pop(G, random_crosses(inbred_founders.haplotypes, 1000))
    founder_pop = create_pop(G, random_crosses(f1.haplotypes, config['pop_size']))
    
    T = Trait(G, founder_pop, target_mean=config['target_mean'], target_variance=config['target_variance'], seed=seed)
    
    SP = SimParams(founder_pop, config)
    env = SelectionIntensityEnvironment(SP, config)
    
    return env




In [ ]:
config = get_default_config()
config['n_parents'] = 2
config['n_loci'] = 50
config['pop_size'] = 100
config['max_generations'] = 10
config['total_timesteps'] = 5000000
config['seed'] = 8
for i in config:
    print(f"{i} : {config[i]}")
env = create_simulation(config)
env.reset()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def collect_baselines(env, actions, repetitions=10, cycles=5):
    results = {action: {'max_phenotype': [], 'gv': []} for action in actions}
    final_gen_averages = {}
    env.reset()
    for action in actions:
        final_gen_phenotypes = []
        for _ in range(repetitions):
            env.reset()
            cycle_max_phenotype = []
            cycle_gv = []
            cycle_max_phenotype.append(env.population.breeding_values.max())
            cycle_gv.append(env.population.breeding_values.var())
            for _ in range(cycles):
                env.step(np.array([action]))
                max_phenotype = env.population.breeding_values.max()
                gv = env.population.breeding_values.var()
                cycle_max_phenotype.append(max_phenotype)
                cycle_gv.append(gv)
            
            results[action]['max_phenotype'].append(cycle_max_phenotype)
            results[action]['gv'].append(cycle_gv)
            final_gen_phenotypes.append(cycle_max_phenotype[-1])
        
        final_gen_averages[action] = np.mean(final_gen_phenotypes)
    
    best_action = max(final_gen_averages, key=final_gen_averages.get)
    best_average = final_gen_averages[best_action]
    
    return results, best_action, best_average

def plot_best_run(results, best_action):
    plt.figure(figsize=(10, 6))
    
    max_phenotypes = np.array(results[best_action]['max_phenotype'])
    mean_values = np.mean(max_phenotypes, axis=0)
    std_values = np.std(max_phenotypes, axis=0)
    cycles = range(len(mean_values))
    best_action_translated = scale_values(best_action)
    
    plt.plot(cycles, mean_values, label=f'Action {best_action_translated:.3f}')
    plt.fill_between(cycles, mean_values - std_values, mean_values + std_values, alpha=0.3)
    
    plt.xlabel('Cycle')
    plt.ylabel('Max Phenotype')
    plt.title(f'Max Phenotype over Cycles for Best Action')
    plt.legend()
    plt.grid(True)
    plt.show()



# Usage
actions = np.linspace(-1, 0, 10)  # or however many actions you want to test
results, best_action, best_average = collect_baselines(env, actions, repetitions=10, cycles=env.SP.max_generations)

print(f"Best action: {best_action:.3f}")
print(f"Best average max phenotype in final generation: {best_average:.3f}")

# Plot only the best run
plot_best_run(results, best_action)


In [ ]:

model = PPO(
    "MlpPolicy",
    env,
    verbose=1,
    tensorboard_log='../dev_nbs/ppotb'
)


# model = PPO(CustomActorCriticPolicy, vec_env, verbose=1, tensorboard_log="./ppotb")
# model.learn(total_timesteps=config['total_timesteps'], callback=combined_callbacks)

# Create your custom callbacks
genetic_variance_callback = AverageFinalGenerationCallback(log_freq=1000)
action_callback = ActionTrackingCallback(log_freq=1000)

# Combine the callbacks using CallbackList
combined_callbacks = CallbackList([genetic_variance_callback, action_callback])
model.learn(total_timesteps=config['total_timesteps'], callback=combined_callbacks)

In [ ]:
plt.hist(env.population.breeding_values)

In [ ]:
env.SP.max_generations

In [ ]:

# Create your custom callbacks
genetic_variance_callback = AverageFinalGenerationCallback(log_freq=1000)
action_callback = ActionTrackingCallback(log_freq=1000)

# Combine the callbacks using CallbackList
combined_callbacks = CallbackList([genetic_variance_callback, action_callback])
vec_env = DummyVecEnv([lambda: env])
# Create and train the model with the custom policy
model = PPO(CustomActorCriticPolicy, vec_env, verbose=1, tensorboard_log="./ppotb")
model.learn(total_timesteps=config['total_timesteps'], callback=combined_callbacks)

In [ ]:
config['sparse_reward'] =True
env = create_simulation(config)
env.reset()
# Create your custom callbacks
genetic_variance_callback = AverageFinalGenerationCallback(log_freq=1000)
action_callback = ActionTrackingCallback(log_freq=1000)

# Combine the callbacks using CallbackList
combined_callbacks = CallbackList([genetic_variance_callback, action_callback])
vec_env = DummyVecEnv([lambda: env])
# Create and train the model with the custom policy
model = PPO(CustomActorCriticPolicy, vec_env, verbose=1, tensorboard_log="./ppotb")
model.learn(total_timesteps=config['total_timesteps'], callback=combined_callbacks)